# Fine tuning of the BAAI/bge-large-en-v1.5 dataset for use in relevance categorization of websites to a topic

In [ ]:
! pip install -U FlagEmbedding[finetune]
!pip install deepspeed
!pip install flash-attn --no-build-isolation


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 97.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.0 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/6

In [ ]:
!git clone https://github.com/FlagOpen/FlagEmbedding.git

Cloning into 'FlagEmbedding'...
remote: Enumerating objects: 10832, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 10832 (delta 182), reused 220 (delta 151), pack-reused 10538 (from 2)
Receiving objects: 100% (10832/10832), 38.93 MiB | 31.71 MiB/s, done.
Resolving deltas: 100% (5945/5945), done.


In [ ]:
#!pip install dbrepo
import json
#get training datatset
from dbrepo.RestClient import RestClient
client = RestClient(endpoint="https://test.dbrepo.tuwien.ac.at", username="11905148", password="qBYh8YylW4l2UsTaaQ3w")


table_data = client.get_table_data(database_id ="e31b2788-e1da-4a4a-9892-d2b5a1216df6", table_id="41c10fad-83e7-4df3-a188-eb872e05646a",size=1000)

data = dict(table_data)

cyber = []
krimisi = []
innotech = []

all_irrelevant = []
print(list(table_data))

for i in range(len(data['bh_category'])):
    if data['is_relevant'][i] == False:
        all_irrelevant.append({"cc_filename": data['cc_filename'][i], "content": data['content'][i], "bh_category": data['bh_category'][i],"cc_normalized_url": data['cc_normalized_url'][i],})

for i in range(len(data['bh_category'])):
    if data['is_relevant'][i] == True:
        row = {"cc_filename": data['cc_filename'][i], "content": data['content'][i], "bh_category": data['bh_category'][i],"cc_normalized_url": data['cc_normalized_url'][i],}
        if "krimisi" in data['bh_category'][i]:
            krimisi.append(row["content"])
            krimisi.append()
        elif "cyber" in data['bh_category'][i]:
            cyber.append(row["content"])
            cyber.append({})
        elif "innotech" in data['bh_category'][i]:
            innotech.append(row["content"])
            innotech.append({})

with open("fine_data.jsonl", "w") as f:
    f.write(json.dumps({"query": "Represent this sentence for searching relevant passages: Articles or news about crises, military or security.","neg": [x["content"] for x in all_irrelevant],"pos": krimisi, "type": "symmetric_class"})+"\n")
    f.write(json.dumps({"query": "Represent this sentence for searching relevant passages: Articles or news about cyber security.","neg": [x["content"] for x in all_irrelevant],"pos": cyber, "type": "symmetric_class"})+"\n")
    f.write(json.dumps({"query": "Represent this sentence for searching relevant passages: Articles or news about new innovations and technology.","neg": [x["content"] for x in all_irrelevant],"pos": innotech, "type": "symmetric_class"})+"\n")
    f.flush()
    




['bh_category', 'cc_filename', 'cc_normalized_url', 'content', 'is_relevant']


In [ ]:
# DO TRAINING
!torchrun --nproc_per_node 1 	-m FlagEmbedding.finetune.embedder.encoder_only.base 	--model_name_or_path BAAI/bge-large-en-v1.5     --cache_dir ./cache/model     --train_data ./fine_data.jsonl --cache_path ./cache/data     --train_group_size 3     --query_max_len 512     --passage_max_len 512     --pad_to_multiple_of 3     --query_instruction_for_retrieval 'Represent this sentence for searching relevant passages: '     --query_instruction_format '{}{}'     --knowledge_distillation False 	--output_dir ./test_encoder_only_base_bge-large-en-v1.5     --overwrite_output_dir     --learning_rate 1e-5     --fp16     --num_train_epochs 2     --per_device_train_batch_size 2     --dataloader_drop_last True     --warmup_ratio 0.1     --gradient_checkpointing     --deepspeed FlagEmbedding/examples/finetune/ds_stage0.json     --logging_steps 1     --save_steps 200     --negatives_cross_device     --temperature 0.02     --sentence_pooling_method cls     --normalize_embeddings True     --kd_loss_type kl_div

2025-04-28 17:48:37.546241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745862517.566928    8954 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745862517.573100    8954 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 17:48:37.594515: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-04-28 17:48:41,690] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025

In [ ]:
import requests
# uplaod model to tuwien model repo

# Set up the API URL and headers
api_url = "https://researchdata.tuwien.at/api/records"
headers = {
    "Authorization": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
}
# todo test_encoder_only_base_bge-large-en-v1.5
# Define the files and metadata
files = {
    'config.json': open('test_encoder_only_base_bge-large-en-v1.5/config.json', 'rb'),
    'model.safetensors': open('test_encoder_only_base_bge-large-en-v1.5/model.safetensors', 'rb'),
    'special_tokens_map.json': open('test_encoder_only_base_bge-large-en-v1.5/special_tokens_map.json', 'rb'),
    'tokenizer_config.json': open('test_encoder_only_base_bge-large-en-v1.5/tokenizer_config.json', 'rb'),
    'tokenizer.json': open('test_encoder_only_base_bge-large-en-v1.5/tokenizer.json', 'rb'),
    'training_args.bin': open('test_encoder_only_base_bge-large-en-v1.5/training_args.bin', 'rb'),
    'vocab.txt': open('test_encoder_only_base_bge-large-en-v1.5/vocab.txt', 'rb'),
}

# Send the POST request to upload
response = requests.post(api_url, headers=headers, files=files)

# Check the response
if response.status_code == 200:
    print("Upload successful")
else:
    print("Failed to upload:", response.text)

    